In [1]:
!pip install transformers datasets

In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, MobileBertForSequenceClassification, AdamW
from datasets import load_dataset

In [3]:
# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import RobertaTokenizer, RobertaForSequenceClassification
# from transformers import AlbertTokenizer, AlbertForSequenceClassification

In [4]:
# Load the FEVER dataset
fever = load_dataset("fever", "v1.0")

# Label mappings
label2id = {"SUPPORTS": 0, "REFUTES": 1, "NOT ENOUGH INFO": 2}
id2label = {v: k for k, v in label2id.items()}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
tokenizer = AutoTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=3)

config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/147M [00:00<?, ?B/s]

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Preprocess data
def preprocess_data(batch):
    inputs = tokenizer(batch["claim"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = [label2id[label] for label in batch["label"]]
    return inputs

In [7]:
len(fever["train"])

311431

In [8]:
len(fever["labelled_dev"])

37566

In [9]:
from collections import Counter

# Count class occurrences in the training dataset
label_counts = Counter(fever["train"]["label"])

# Total number of samples
total_samples = len(fever["train"]["label"])

# Calculate and print the percentage for each class
for label, count in label_counts.items():
    percentage = (count / total_samples) * 100
    print(f"Class '{label}': {count} samples, {percentage:.2f}%")


Class 'SUPPORTS': 193756 samples, 62.21%
Class 'REFUTES': 70066 samples, 22.50%
Class 'NOT ENOUGH INFO': 47609 samples, 15.29%


In [10]:
from datasets import Dataset

# Function to sample equal examples for each class
def sample_equal_classes(dataset, label_column, num_samples_per_class):
    sampled_data = []
    for label in label2id.values():  # Iterate over label IDs
        class_samples = dataset.filter(lambda x: label2id[x[label_column]] == label)
        sampled_data.append(class_samples.select(range(min(num_samples_per_class, len(class_samples)))))
    # Combine all class samples
    return Dataset.from_dict({key: sum([d[key] for d in sampled_data], []) for key in dataset.column_names})

# Sample 20,000 training examples and 5,000 validation examples
train_data = sample_equal_classes(fever["train"], "label", num_samples_per_class=10000 // len(label2id))
val_data = sample_equal_classes(fever["labelled_dev"], "label", num_samples_per_class=1000 // len(label2id))

# Preprocess the data
train_data = train_data.map(preprocess_data, batched=True)
val_data = val_data.map(preprocess_data, batched=True)

# Convert to PyTorch format
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Create DataLoaders
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=32)

print(f"Training examples: {len(train_data)}, Validation examples: {len(val_data)}")

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Training examples: 9999, Validation examples: 999


In [11]:
# from collections import Counter

# # Count class occurrences in the training dataset
# label_counts = Counter(train_data["label"])

# # Total number of samples
# total_samples = len(train_data["label"])

# # Calculate and print the percentage for each class
# for label, count in label_counts.items():
#     percentage = (count / total_samples) * 100
#     print(f"Class '{label}': {count} samples, {percentage:.2f}%")


In [12]:
# train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
# val_dataloader = DataLoader(val_data, batch_size=128)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


MobileBertForSequenceClassification(
  (mobilebert): MobileBertModel(
    (embeddings): MobileBertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
      (LayerNorm): NoNorm()
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): MobileBertEncoder(
      (layer): ModuleList(
        (0-23): 24 x MobileBertLayer(
          (attention): MobileBertAttention(
            (self): MobileBertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=512, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MobileBertSelfOutput(
              (dense): Linear(in_fe

In [14]:
from tqdm import tqdm

for epoch in range(5):  # Number of epochs
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    # Wrap the DataLoader with TQDM for a progress bar
    train_progress = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")

    for batch in train_progress:
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)

        # print("Input shapes:", {k: v.shape for k, v in inputs.items()})
        # print("Label shape:", labels.shape)

        # Forward pass
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate predictions and update accuracy metrics
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

        # Update TQDM with the current batch loss and accuracy
        train_progress.set_postfix(loss=loss.item(), accuracy=correct_predictions / total_predictions)

    # Calculate epoch-level accuracy
    epoch_accuracy = correct_predictions / total_predictions

    # Print epoch-level loss and accuracy
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader):.4f}, Accuracy: {epoch_accuracy:.4f}")

Epoch 1: 100%|██████████| 313/313 [01:49<00:00,  2.86it/s, accuracy=0.333, loss=259]


Epoch 1, Loss: 113549.9859, Accuracy: 0.3326


Epoch 2: 100%|██████████| 313/313 [01:36<00:00,  3.24it/s, accuracy=0.332, loss=1.71]


Epoch 2, Loss: 592.9715, Accuracy: 0.3317


Epoch 3: 100%|██████████| 313/313 [01:37<00:00,  3.20it/s, accuracy=0.336, loss=1.16]


Epoch 3, Loss: 93.9719, Accuracy: 0.3356


Epoch 4: 100%|██████████| 313/313 [01:38<00:00,  3.18it/s, accuracy=0.331, loss=1.2]


Epoch 4, Loss: 67.6520, Accuracy: 0.3312


Epoch 5: 100%|██████████| 313/313 [01:38<00:00,  3.19it/s, accuracy=0.332, loss=1.23]

Epoch 5, Loss: 30.7780, Accuracy: 0.3318


In [15]:
from sklearn.metrics import accuracy_score, classification_report


model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)

        outputs = model(**inputs)
        logits = outputs.logits

        # Get predictions (the index of the highest logit)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"]))

Validation Accuracy: 0.2673
Classification Report:
                 precision    recall  f1-score   support

       SUPPORTS       0.26      0.43      0.32       333
        REFUTES       0.30      0.06      0.10       333
NOT ENOUGH INFO       0.27      0.32      0.29       333

       accuracy                           0.27       999
      macro avg       0.28      0.27      0.24       999
   weighted avg       0.28      0.27      0.24       999



In [16]:
# Function to predict the class of a particular claim
def predict_claim_class(claim):
    # Preprocess the claim (same way as during training)
    inputs = tokenizer(claim, padding="max_length", truncation=True, max_length=128, return_tensors="pt").to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (index of max logit)
    predicted_class_id = torch.argmax(logits, dim=1).item()

    # Convert class ID to label
    predicted_label = id2label[predicted_class_id]

    return predicted_label


In [17]:
# Example usage:
claim = "The Atlantic Ocean is the largest ocean on Earth."
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'The Atlantic Ocean is the largest ocean on Earth.' belongs to the class: SUPPORTS


In [18]:
# Example usage:
claim= "FIFA World Cup in 2022 was won by France National team."
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'FIFA World Cup in 2022 was won by France National team.' belongs to the class: NOT ENOUGH INFO


In [19]:
# Example usage:
claim="The sun appears yellow when observed from the Earth "
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'The sun appears yellow when observed from the Earth ' belongs to the class: NOT ENOUGH INFO


In [20]:
# Example usage:
claim= "The Earth's shadow on the moon is approximately 14,500 miles"
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'The Earth's shadow on the moon is approximately 14,500 miles' belongs to the class: NOT ENOUGH INFO


In [21]:
# Example usage:
claim="World War 2 began on September 1, 939, when Nazi Germany invaded Poland."
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'World War 2 began on September 1, 939, when Nazi Germany invaded Poland.' belongs to the class: SUPPORTS


In [22]:
# Example usage:
claim="India gained independence on August 15, 2007, after being ruled by the British for nearly 30 years."
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'India gained independence on August 15, 2007, after being ruled by the British for nearly 30 years.' belongs to the class: SUPPORTS


In [23]:
# Example usage:
claim="The Eiffel tower was designed by Gustave Eiffele, a French engineer and archite"
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'The Eiffel tower was designed by Gustave Eiffele, a French engineer and archite' belongs to the class: SUPPORTS


In [24]:
# Example usage:
claim="The first practical telecommunications device was invent by Alexander Graham Bell"
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'The first practical telecommunications device was invent by Alexander Graham Bell' belongs to the class: SUPPORTS


In [25]:
# Example usage:
claim="The Nobel prize in literature in the year 2024 was awarded to Margaret Atwood"
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'The Nobel prize in literature in the year 2024 was awarded to Margaret Atwood' belongs to the class: SUPPORTS
